In [57]:
import os
import f90nml

# update the parameter values in ["nml_site_params", "nml_species_params"] to mcmc_conf

def handle_data(iplot):
    # Define file paths
    parameters_nml_path = f"inputs/in_treat/{iplot}/parameters.nml"
    mcmc_conf_nml_path  = "configs/mcmc_conf.nml"

    # Read parameters.nml content
    nml_params_data    = f90nml.read(parameters_nml_path)
    nml_mcmc_conf_data = f90nml.read(mcmc_conf_nml_path)

    # dict paramter
    dict_site_params = dict(nml_params_data["nml_site_params"])
    dict_spp_params  = dict(nml_params_data["nml_species_params"])

    # modify mcmc_conf data
    for idx_nml, inml in enumerate(nml_mcmc_conf_data["siteDAParams"]["st"]):
        if inml["parname"] in dict_site_params.keys():
            nml_mcmc_conf_data["siteDAParams"]["st"][idx_nml]["parVal"] = dict_site_params[inml["parname"]]


    for idx_sp, isp in enumerate(nml_mcmc_conf_data["spDAParams"]["sp"]):
        for idx_var, ivar in enumerate(isp["var"]):
            if ivar["parname"] in dict_spp_params.keys():
                nml_mcmc_conf_data["spDAParams"]["sp"][idx_sp]["var"][idx_var]["parval"] = dict_spp_params[ivar["parname"]][idx_sp]

    # write updated mcmc_conf to file
    nml_mcmc_conf_data.write(f"mcmc_conf_{iplot}.nml")

ls_plots = ["P04", "P06", "P08", "P10", "P11", "P13", "P16", "P17", "P19", "P20"]

for idx, iplot in enumerate(ls_plots):
    handle_data(iplot)


# # 递归写入 NAMELIST 数据
# def write_namelist(nml_data, file):
#     """ 递归地写入 namelist 数据到文件 """
#     for group, params in nml_data.items():
#         file.write(f"&{group}\n")  # 写入 namelist 组名
#         write_parameters(params, file)  # 递归处理参数
#         file.write("/\n\n")  # 结束 namelist 组

# def write_parameters(params, file, prefix=""):
#     """ 递归写入 namelist 参数 """
#     for key, value in params.items():
#         if isinstance(value, dict):  # 处理嵌套字典
#             write_parameters(value, file, prefix=f"{prefix}{key} % ")
#         elif isinstance(value, list):
#             if all(isinstance(v, dict) for v in value):  # 处理 list of dicts
#                 for i, subdict in enumerate(value, start=1):
#                     write_parameters(subdict, file, prefix=f"{prefix}{key}({i}) % ")
#             else:  # 处理普通列表
#                 value_str = ', '.join(map(str, value))
#                 file.write(f"  {prefix}{key} = {value_str}\n")
#         elif isinstance(value, str):  # 处理字符串
#             file.write(f"  {prefix}{key} = '{value}'\n")
#         else:  # 处理数值
#             file.write(f"  {prefix}{key} = {value}\n")

# # 逐行写入新的 nml 文件
# with open('output.nml', 'w') as f:
#     write_namelist(nml_mcmc_conf_data, f)



In [ ]:


# with open(parameters_nml_path, "r") as f:
#     parameters_nml_content = f.readlines()

# # Read mcmc_conf.nml content
# with open(mcmc_conf_nml_path, "r") as f:
#     mcmc_conf_nml_content = f.readlines()

# print(parameters_nml_content)
# print(mcmc_conf_nml_content)

# # Extract site parameters and species parameters from parameters.nml
# site_parameters_dict = {}
# species_parameters_dict = {}

# for line in parameters_nml_content:
#     if "=" in line and "!" not in line:  # Ignore comments
#         parts = line.split("=")
#         key = parts[0].strip()
#         values = parts[1].strip().split()

#         try:
#             if len(values) == 1:  # Single value (site parameter)
#                 site_parameters_dict[key] = float(values[0])
#             elif len(values) == 3 and key.startswith("def_"):  # Species parameter
#                 species_parameters_dict[key] = [float(v) for v in values]
#         except ValueError:
#             continue  # Skip invalid values

# # Modify mcmc_conf.nml content
# updated_mcmc_conf_content = []
# for line in mcmc_conf_nml_content:
#     # Update general site parameters
#     if "parName" in line and "parVal" in line:
#         parts = line.split(",")
#         par_name_part = parts[0].split("=")
#         par_val_part = parts[1].split("=")

#         par_name = par_name_part[1].strip().strip('"')
#         if par_name in site_parameters_dict:
#             new_value = site_parameters_dict[par_name]
#             new_line = f'{par_name_part[0]}= "{par_name}", {par_val_part[0]}= {new_value:.5f},' + ",".join(parts[2:])
#             updated_mcmc_conf_content.append(new_line)
#         else:
#             updated_mcmc_conf_content.append(line)

#     # Update species parameters (sp(n)%var(m)%parVal)
#     elif "sp(" in line and "var(" in line and "parVal" in line:
#         parts = line.split("=")
#         var_index = parts[0].split("var(")[-1].split(")")[0]
#         sp_index = parts[0].split("sp(")[-1].split(")")[0]

#         try:
#             param_value = float(parts[1].strip())
#             species_key = f"def_{var_index}"  # Assuming def_xxx maps to var(xxx)
#             if species_key in species_parameters_dict:
#                 new_value = species_parameters_dict[species_key][int(sp_index) - 1]  # Adjust for 1-based index
#                 new_line = f'{parts[0]}= {new_value:.5f}\n'
#                 updated_mcmc_conf_content.append(new_line)
#             else:
#                 updated_mcmc_conf_content.append(line)
#         except ValueError:
#             updated_mcmc_conf_content.append(line)

#     else:
#         updated_mcmc_conf_content.append(line)

# # Function to align assignments in Fortran namelist format
# def align_nml_assignments(lines):
#     max_eq_index = max(line.find("=") for line in lines if "=" in line)
#     aligned_lines = []
    
#     for line in lines:
#         if "=" in line:
#             parts = line.split("=")
#             left_part = parts[0].rstrip()
#             right_part = "=" + parts[1].lstrip()
#             aligned_line = f"{left_part.ljust(max_eq_index)} {right_part}"
#             aligned_lines.append(aligned_line)
#         else:
#             aligned_lines.append(line)
    
#     return aligned_lines

# # Align the modified content
# aligned_mcmc_conf_content = align_nml_assignments(updated_mcmc_conf_content)

# print(aligned_mcmc_conf_content)

# # # Write the aligned content back to the file
# # with open(updated_mcmc_conf_path, "w") as f:
# #     f.writelines(aligned_mcmc_conf_content)

# # print(f"Updated and aligned file saved at: {updated_mcmc_conf_path}")